In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from functools import reduce
from scipy.optimize import fsolve

In [2]:
def full_soln(x):
    return list(x) + [1 - sum(x[1:])]

In [3]:
NE_data = dict()
for N in range(3, 5):
    start_time = datetime.now()
    print("N = ", N)
    print("Start Time: ", start_time)
    
    p = sp.IndexedBase('p')
    i = sp.symbols('i', cls=sp.Idx, range=(1,N))
    Z0, c0 = sp.symbols('Z0, c0')
    Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
    L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
    OneMinusL = lambda i: lambda Q: Q - L(i)(Q)
    Z = lambda i: Z0 if i==0 else reduce(lambda f,g: lambda x: f(g(x)), [OneMinusL(x) for x in range(1, i+1)])(Z0)
    c = lambda i: Z(i-1).subs(p[i], 0)
    func = lambda x: [sp.lambdify([c0]+[p[i] for i in range(1, N)], c(j).subs(p[N], 1-sp.summation(p[i], (i, 1, N-1))).simplify()-c0)(*x) for j in range(1, N+1)]
    soln = full_soln(fsolve(func, [1/N for _ in range(N)]))
    print(soln)
    NE_data[N] = soln
    # with open("data/NE{}.pk".format(N), 'wb') as file:
    #     pickle.dump(soln, file)
    
    end_time = datetime.now()
    print("End Time: ", end_time)
    diff = end_time - start_time
    print("Elapsed Time: {} days {} hours {} minutes {} seconds {} milliseconds {} microseconds".format(diff.days, diff.seconds // 3600, (diff.seconds // 60) % 60, diff.seconds % 60, diff.microseconds // 1000, diff.microseconds % 1000))
    print()

N =  3
Start Time:  2024-04-25 01:06:37.070018
[0.28718707889821465, 0.4641016151377231, 0.2679491924341272, 0.26794919242814974]
End Time:  2024-04-25 01:06:37.502620
Elapsed Time: 0 days 0 hours 0 minutes 0 seconds 432 milliseconds 602 microseconds

N =  4
Start Time:  2024-04-25 01:06:37.502620
[0.1684375244852117, 0.4477365732247908, 0.4248727049531577, 0.12565488835943256, 0.001735833462618963]
End Time:  2024-04-25 01:06:38.739631
Elapsed Time: 0 days 0 hours 0 minutes 1 seconds 237 milliseconds 11 microseconds



In [4]:
N = 3
p = sp.IndexedBase('p')
i,j = sp.symbols('i j', cls=sp.Idx, range=(1,N))
Z0, c0 = sp.symbols('Z0, c0')
Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
OneMinusL = lambda i: lambda Q: Q - L(i)(Q)
Z = lambda i: Z0 if i==0 else reduce(lambda f,g: lambda x: f(g(x)), [OneMinusL(x) for x in range(1, i+1)])(Z0)
c = lambda i: Z(i-1).subs(p[i], 0)

In [5]:
for i in range(1, N+1):
    display(c(i).subs(p[N], 1-sp.summation(p[j], (j, 1, N-1))))

(1 - p[1])**2

(1 - p[2])**2 - 2*(-p[1] - p[2] + 1)*p[1]

(p[1] + p[2])**2 - 2*p[1]*p[2]

In [9]:
c0 = 0.287
ps = dict()
for n in range(1, N):
    func = c(n).subs(p[N], 1-sp.summation(p[j], (j, 1, N-1)))-c0
    display(func)
    func = func.subs(ps)
    ps[p[n]] = sp.solve(func, p[n])[0]
    display(func)
    print(ps[p[n]])
func = c(N).subs(p[1], 1-sp.summation(p[j], (j, 2, N)))-c0
display(func)
func = func.subs(ps)
display(func)
pN = sp.solve(func, p[N])
print(pN)

(1 - p[1])**2 - 0.287

(1 - p[1])**2 - 0.287

0.464276190560845


(1 - p[2])**2 - 2*(-p[1] - p[2] + 1)*p[1] - 0.287

(1 - p[2])**2 + 0.92855238112169*p[2] - 0.78444761887831

0.268426935386520


(1 - p[3])**2 - 2*(-p[2] - p[3] + 1)*p[2] - 0.287

(1 - p[3])**2 + 0.536853870773041*p[3] - 0.679747831491042

[0.267949315913092, 1.19519681331387]


In [7]:
c(N)

(p[1] + p[2])**2 - 2*p[1]*p[2]